In [1]:
from hdmm.workload import *
from hdmm import templates, error
%load_ext autoreload
%autoreload 2

In [39]:
n = 128
P = Prefix(n)
T = Total(n)
I = Identity(n)
S = EkteloMatrix(np.kron(np.eye(4), np.ones(n // 4)))
K = lambda *A: Kronecker(A)

W1 = VStack([K(P,T), K(T, P)])
W2 = VStack([K(P, S), K(S, P)])
W3 = VStack([K(P,P,T),K(P,T,P), K(T,P,P)])
W4 = VStack([K(P,P,T), K(P,T,P), K(T,P,P), K(P,T,T), K(T,P,T), K(T,T,P)])
W5 = VStack([K(I,I,T), K(I,T,I), K(T,I,I)])

In [40]:
for W in [W1, W2, W3, W4, W5]:
    d = len(W.matrices[0].matrices)
    p = n//16
    optx = templates.KronPIdentity([p]*d, [n]*d)
    optx.optimize(W)
    A = optx.strategy()
    error1 = error.rootmse(W, A)
    
    ps = []
    for Wi in W.matrices:
        ps.append(tuple(1 if Q == T else p for Q in Wi.matrices))
    optp = templates.UnionKron(ps, [n]*d)
    optp.optimize(W.matrices)
    
    error2 = 0
    for Wi, Ai in zip(W.matrices, optp.strategy().matrices):
        w, A = Ai.weight, Ai.base
        error2 += error.expected_error(Wi, A) / w**2
    error2 = np.sqrt(error2 / W.shape[0])
    
    optm = templates.Marginals([n]*d)
    optm.optimize(W)
    error3 = optm._loss_and_grad(optm._params)[0] * n**d
    error3 = np.sqrt(error3 / W.shape[0])
    
    print(error1, error2, error3)

14.39030429003844 8.994703513917983 11.381295819255488
10.16515134408462 12.321383254830337 38.05277676382285
83.60040364635458 60.38831768164826 98.67828341649532
80.90127541207043 67.26306111669037 98.49002758095769
17.857277056913656 3.009599456647092 2.988292586405388


In [33]:
optp._weights

array([0.30939335, 0.31152226, 0.3108255 , 0.02268439, 0.02286453,
       0.02270996])